In [ ]:
import random
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from typing import List, Tuple, Dict


In [ ]:
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(6, 16, 5, stride=1, padding=0)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)

        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)

        output = F.log_softmax(x, dim=1)

        return output

In [ ]:
class CompactCNN(nn.Module):
    """
    Realistic architecture an attacker might use knowing the task is digit recognition.
    Based on common MNIST tutorial patterns.
    """
    def __init__(self):
        super(CompactCNN, self).__init__()
        # Common MNIST CNN pattern: 32->64 channels
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        # Common MNIST FC sizes: 128->10
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

In [ ]:
class ModelFingerprinting:
    def __init__(self, model: nn.Module, device: str = 'cpu'):
        """
        Initialize model fingerprinting defense
        
        Args:
            model: The protected model
            device: Device to run computations on
        """
        self.model = model
        self.device = device
        self.model.to(device)
        self.fingerprint_samples = []
        self.fingerprint_labels = []
        
    def generate_boundary_samples(self, 
                                data_loader: torch.utils.data.DataLoader,
                                num_samples: int = 100,
                                num_iterations: int = 10,
                                step_size: float = 0.01) -> List[Tuple[torch.Tensor, int]]:
        """
        Generate fingerprint samples near decision boundaries using adversarial perturbations
        
        Args:
            data_loader: DataLoader with training/test data
            num_samples: Number of fingerprint samples to generate
            num_iterations: Number of iterations for boundary search
            step_size: Step size for gradient-based boundary search
            
        Returns:
            List of (sample, label) tuples
        """
        print(f"Generating {num_samples} boundary fingerprint samples...")
        
        self.model.eval()
        fingerprint_data = []
        samples_generated = 0
        
        with torch.enable_grad():
            for batch_idx, (data, target) in enumerate(data_loader):
                if samples_generated >= num_samples:
                    break
                    
                data, target = data.to(self.device), target.to(self.device)
                
                for i in range(min(len(data), num_samples - samples_generated)):
                    original_sample = data[i:i+1].clone()
                    original_label = target[i].item()
                    
                    # Generate boundary sample using targeted adversarial attack
                    boundary_sample = self._find_boundary_sample(
                        original_sample, original_label, num_iterations, step_size
                    )
                    
                    if boundary_sample is not None:
                        # Get the label for the boundary sample
                        with torch.no_grad():
                            output = self.model(boundary_sample)
                            boundary_label = output.argmax(dim=1).item()
                        
                        # Remove batch dimension before storing
                        fingerprint_data.append((boundary_sample.squeeze(0).cpu(), boundary_label))
                        samples_generated += 1
                        
                        if samples_generated % 10 == 0:
                            print(f"Generated {samples_generated}/{num_samples} samples")
        
        print(f"Successfully generated {len(fingerprint_data)} fingerprint samples")
        return fingerprint_data
    
    def _find_boundary_sample(self, 
                            sample: torch.Tensor, 
                            original_label: int,
                            num_iterations: int, 
                            step_size: float) -> torch.Tensor:
        """
        Find a sample near the decision boundary using gradient-based search
        """
        # Try to find boundary with different target classes
        target_classes = [i for i in range(10) if i != original_label]
        random.shuffle(target_classes)
        
        for target_class in target_classes[:3]:  # Try top 3 different classes
            perturbed_sample = sample.clone().detach().requires_grad_(True)
            
            for iteration in range(num_iterations):
                output = self.model(perturbed_sample)
                
                # Loss to move towards target class
                target_tensor = torch.tensor([target_class], device=self.device)
                loss = F.cross_entropy(output, target_tensor)
                
                # Compute gradients
                self.model.zero_grad()
                loss.backward()
                
                # Update sample towards boundary
                with torch.no_grad():
                    grad_sign = perturbed_sample.grad.sign()
                    perturbed_sample -= step_size * grad_sign
                    
                    # Clamp to valid pixel range (assuming normalized MNIST)
                    perturbed_sample.clamp_(-2.5, 2.5)  # Approximate range for normalized MNIST
                
                perturbed_sample.grad.zero_()
                
                # Check if we're near boundary (confidence close to 0.5 for binary decision)
                with torch.no_grad():
                    output = self.model(perturbed_sample)
                    probs = F.softmax(output, dim=1)
                    max_prob = probs.max().item()
                    
                    # If we found a sample with uncertain prediction, return it
                    if 0.3 < max_prob < 0.7:  # Near decision boundary
                        return perturbed_sample.detach()
        
        return None
    
    def create_fingerprints(self, 
                          data_loader: torch.utils.data.DataLoader,
                          num_samples: int = 100,
                          save_path: str = None) -> Dict:
        """
        Create fingerprints for the model (Offline Phase)
        
        Args:
            data_loader: DataLoader with training/test data
            num_samples: Number of fingerprint samples to generate
            save_path: Path to save fingerprints
            
        Returns:
            Dictionary containing fingerprint data
        """
        print("=== OFFLINE PHASE: Creating Model Fingerprints ===")
        
        # Generate boundary samples
        fingerprint_data = self.generate_boundary_samples(data_loader, num_samples)
        
        # Store fingerprints
        self.fingerprint_samples = [sample for sample, _ in fingerprint_data]
        self.fingerprint_labels = [label for _, label in fingerprint_data]
        
        fingerprint_dict = {
            'samples': self.fingerprint_samples,
            'labels': self.fingerprint_labels,
            'num_samples': len(self.fingerprint_samples)
        }
        
        # Save fingerprints if path provided
        if save_path:
            with open(save_path, 'wb') as f:
                pickle.dump(fingerprint_dict, f)
            print(f"Fingerprints saved to {save_path}")
        
        print(f"Created {len(self.fingerprint_samples)} fingerprint samples")
        return fingerprint_dict
    
    def verify_model(self, 
                    suspect_model: nn.Module, 
                    threshold: float = 0.8,
                    fingerprint_path: str = None) -> Tuple[bool, float, Dict]:
        """
        Verify if a suspect model is a pirated version (Online Phase)
        
        Args:
            suspect_model: The model to verify
            threshold: Matching rate threshold for detection
            fingerprint_path: Path to load fingerprints from
            
        Returns:
            (is_pirated, matching_rate, detailed_results)
        """
        print("=== ONLINE PHASE: Verifying Suspect Model ===")
        
        # Load fingerprints if path provided
        if fingerprint_path:
            with open(fingerprint_path, 'rb') as f:
                fingerprint_dict = pickle.load(f)
            self.fingerprint_samples = fingerprint_dict['samples']
            self.fingerprint_labels = fingerprint_dict['labels']
        
        if not self.fingerprint_samples:
            raise ValueError("No fingerprints available. Run create_fingerprints first.")
        
        suspect_model.to(self.device)
        suspect_model.eval()
        
        matches = 0
        total_samples = len(self.fingerprint_samples)
        detailed_results = []
        
        print(f"Testing {total_samples} fingerprint samples...")
        
        with torch.no_grad():
            for i, (sample, original_label) in enumerate(zip(self.fingerprint_samples, self.fingerprint_labels)):
                sample = sample.to(self.device)
                
                # Ensure sample has correct dimensions [1, 1, 28, 28] for MNIST
                if len(sample.shape) == 3:  # [1, 28, 28]
                    sample = sample.unsqueeze(0)  # Add batch dimension -> [1, 1, 28, 28]
                elif len(sample.shape) == 2:  # [28, 28]
                    sample = sample.unsqueeze(0).unsqueeze(0)  # -> [1, 1, 28, 28]
                
                # Get prediction from suspect model
                suspect_output = suspect_model(sample)
                suspect_label = suspect_output.argmax(dim=1).item()
                
                # Check if labels match
                is_match = (suspect_label == original_label)
                if is_match:
                    matches += 1
                
                detailed_results.append({
                    'sample_idx': i,
                    'original_label': original_label,
                    'suspect_label': suspect_label,
                    'match': is_match
                })
                
                if (i + 1) % 20 == 0:
                    print(f"Processed {i + 1}/{total_samples} samples")
        
        matching_rate = matches / total_samples
        is_pirated = matching_rate >= threshold
        
        print(f"\n=== VERIFICATION RESULTS ===")
        print(f"Total fingerprint samples: {total_samples}")
        print(f"Matching predictions: {matches}")
        print(f"Matching rate: {matching_rate:.3f}")
        print(f"Threshold: {threshold:.3f}")
        print(f"Verdict: {'PIRATED MODEL DETECTED' if is_pirated else 'Model appears legitimate'}")
        
        return is_pirated, matching_rate, {
            'total_samples': total_samples,
            'matches': matches,
            'matching_rate': matching_rate,
            'threshold': threshold,
            'detailed_results': detailed_results
        }

In [2]:
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [47]:
# Load MNIST data
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST("../data", train=True, download=True, transform=transform)

# 50:50 for Lenet:CompCNN train datasets
lenet_train_size = int(0.5 * len(train_dataset))
compcnn_train_size = len(train_dataset) - lenet_train_size
lenet_train_dataset, compcnn_train_dataset = random_split(train_dataset, [lenet_train_size, compcnn_train_size])

# test dataset for evaluation
test_dataset = datasets.MNIST("../data", train=False, transform=transform)

In [48]:
# Set up data loader
lenet_train_loader = torch.utils.data.DataLoader(lenet_train_dataset, batch_size=64, shuffle=True)
compcnn_train_loader = torch.utils.data.DataLoader(compcnn_train_dataset, batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [10]:
# Build the Lenet model
lenet_model = Lenet()

# Define the optimizer for model training
optimizer = optim.Adadelta(lenet_model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

lenet_model.train()

for epoch in range(1, 6):
    for batch_idx, (data, target) in enumerate(lenet_train_loader):
        optimizer.zero_grad()
        output = lenet_model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(lenet_train_loader.dataset),
                    100.0 * batch_idx / len(lenet_train_loader),
                    loss.item(),
                )
            )

    scheduler.step()

Train Epoch: 1 [0/30000 (0%)]	Loss: 2.299041
Train Epoch: 1 [640/30000 (2%)]	Loss: 2.289697
Train Epoch: 1 [1280/30000 (4%)]	Loss: 2.031714
Train Epoch: 1 [1920/30000 (6%)]	Loss: 1.758418
Train Epoch: 1 [2560/30000 (9%)]	Loss: 1.493877
Train Epoch: 1 [3200/30000 (11%)]	Loss: 0.871744
Train Epoch: 1 [3840/30000 (13%)]	Loss: 0.921750
Train Epoch: 1 [4480/30000 (15%)]	Loss: 0.570331
Train Epoch: 1 [5120/30000 (17%)]	Loss: 0.856902
Train Epoch: 1 [5760/30000 (19%)]	Loss: 0.504076
Train Epoch: 1 [6400/30000 (21%)]	Loss: 0.613116
Train Epoch: 1 [7040/30000 (23%)]	Loss: 0.298030
Train Epoch: 1 [7680/30000 (26%)]	Loss: 0.562277
Train Epoch: 1 [8320/30000 (28%)]	Loss: 0.735044
Train Epoch: 1 [8960/30000 (30%)]	Loss: 0.435347
Train Epoch: 1 [9600/30000 (32%)]	Loss: 0.508466
Train Epoch: 1 [10240/30000 (34%)]	Loss: 0.326847
Train Epoch: 1 [10880/30000 (36%)]	Loss: 0.541892
Train Epoch: 1 [11520/30000 (38%)]	Loss: 0.587256
Train Epoch: 1 [12160/30000 (41%)]	Loss: 0.334622
Train Epoch: 1 [12800/300

In [11]:
# Evaluate the model
lenet_model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        output = lenet_model(data)
        test_loss += F.nll_loss(
            output, target, reduction="sum"
        ).item()  # sum up batch loss

        pred = output.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability

        correct += pred.eq(target.view_as(pred)).sum().item()


test_loss /= len(test_loader.dataset)

print(
    "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        test_loss,
        correct,
        len(test_loader.dataset),
        100.0 * correct / len(test_loader.dataset),
    )
)


Test set: Average loss: 0.0579, Accuracy: 9840/10000 (98%)



In [12]:
# Save the model
torch.save(lenet_model.state_dict(), "mnist_lenet_cnn.pt")

In [13]:
# Build the CompactCNN model
comp_cnn_model = CompactCNN()

# Define the optimizer for model training
optimizer = optim.Adadelta(comp_cnn_model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

comp_cnn_model.train()

for epoch in range(1, 6):
    for batch_idx, (data, target) in enumerate(compcnn_train_loader):
        optimizer.zero_grad()
        output = comp_cnn_model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(compcnn_train_loader.dataset),
                    100.0 * batch_idx / len(compcnn_train_loader),
                    loss.item(),
                )
            )

    scheduler.step()

Train Epoch: 1 [0/30000 (0%)]	Loss: 2.296233
Train Epoch: 1 [640/30000 (2%)]	Loss: 1.768871
Train Epoch: 1 [1280/30000 (4%)]	Loss: 0.782838
Train Epoch: 1 [1920/30000 (6%)]	Loss: 0.607692
Train Epoch: 1 [2560/30000 (9%)]	Loss: 0.395882
Train Epoch: 1 [3200/30000 (11%)]	Loss: 0.296795
Train Epoch: 1 [3840/30000 (13%)]	Loss: 0.172108
Train Epoch: 1 [4480/30000 (15%)]	Loss: 0.299837
Train Epoch: 1 [5120/30000 (17%)]	Loss: 0.241298
Train Epoch: 1 [5760/30000 (19%)]	Loss: 0.581469
Train Epoch: 1 [6400/30000 (21%)]	Loss: 0.172319
Train Epoch: 1 [7040/30000 (23%)]	Loss: 0.266466
Train Epoch: 1 [7680/30000 (26%)]	Loss: 0.447399
Train Epoch: 1 [8320/30000 (28%)]	Loss: 0.166679
Train Epoch: 1 [8960/30000 (30%)]	Loss: 0.352879
Train Epoch: 1 [9600/30000 (32%)]	Loss: 0.345815
Train Epoch: 1 [10240/30000 (34%)]	Loss: 0.240938
Train Epoch: 1 [10880/30000 (36%)]	Loss: 0.167653
Train Epoch: 1 [11520/30000 (38%)]	Loss: 0.102553
Train Epoch: 1 [12160/30000 (41%)]	Loss: 0.417771
Train Epoch: 1 [12800/300

In [14]:
# Evaluate the model
comp_cnn_model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        output = comp_cnn_model(data)
        test_loss += F.nll_loss(
            output, target, reduction="sum"
        ).item()  # sum up batch loss

        pred = output.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability

        correct += pred.eq(target.view_as(pred)).sum().item()


test_loss /= len(test_loader.dataset)

print(
    "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        test_loss,
        correct,
        len(test_loader.dataset),
        100.0 * correct / len(test_loader.dataset),
    )
)


Test set: Average loss: 0.0289, Accuracy: 9901/10000 (99%)



In [15]:
# Save the model
torch.save(comp_cnn_model.state_dict(), "mnist_comp_cnn.pt")

In [39]:
# Set up MEA parameters
attack_number = 3000  # maximum attack_number is 30 000
attack_indices = random.sample(range(0, len(lenet_train_dataset)), attack_number)

queries = torch.utils.data.Subset(lenet_train_dataset, attack_indices)
queries_loader = torch.utils.data.DataLoader(queries, batch_size=64, shuffle=True)

In [40]:
# Instantiate extracted model using realistic attacker architecture
extracted_model = CompactCNN()

# Define the optimizer for model extracting (optimized for knowledge distillation)
optimizer = optim.Adam(extracted_model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=25, gamma=0.5)

# Perform model extraction
lenet_model.eval()

for epoch in range(20):
    epoch_loss = 0
    num_batches = 0

    for data, target in tqdm(queries_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        output = extracted_model(data)

        # Query target model without computing gradients
        with torch.no_grad():
            target_output = lenet_model(data)

        loss = F.kl_div(F.log_softmax(output, dim=1), F.softmax(target_output, dim=1), reduction='batchmean', log_target=False)
        loss.backward()
        optimizer.step()

        # Update running statistics
        epoch_loss += loss.item()
        num_batches += 1

    print(f"Avg Loss: {epoch_loss/num_batches:.4f}")

    scheduler.step()

print("\n" + "=" * 50)
print("Model extraction completed!")

Epoch 1: 100%|██████████| 47/47 [00:12<00:00,  3.90it/s]


Avg Loss: 1.1324


Epoch 2: 100%|██████████| 47/47 [00:09<00:00,  4.76it/s]


Avg Loss: 0.3811


Epoch 3: 100%|██████████| 47/47 [00:04<00:00, 10.05it/s]


...

Avg Loss: 0.0436


Epoch 19: 100%|██████████| 47/47 [00:05<00:00,  8.80it/s]


Avg Loss: 0.0434


Epoch 20: 100%|██████████| 47/47 [00:05<00:00,  8.79it/s]

Avg Loss: 0.0441

Model extraction completed!


In [41]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

extracted_model.eval()

test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in tqdm(test_loader):
        output = extracted_model(data)
        test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print(
    "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        test_loss,
        correct,
        len(test_loader.dataset),
        100.0 * correct / len(test_loader.dataset),
    )
)

100%|██████████| 157/157 [00:09<00:00, 17.03it/s]


Test set: Average loss: 0.0778, Accuracy: 9765/10000 (98%)



In [42]:
# Save the model
torch.save(extracted_model.state_dict(), "mnist_comp_cnn_mea_10.pt")

In [4]:
# Initialize the original model
original_model = Lenet()

original_model.load_state_dict(torch.load('mnist_lenet_cnn.pt', map_location=device))
print("Loaded trained model successfully")

Loaded trained model successfully


In [5]:
# Initialize fingerprinting system
fingerprinting = ModelFingerprinting(original_model, device)

In [22]:
# OFFLINE PHASE: Create fingerprints
fingerprints = fingerprinting.create_fingerprints(
    test_loader, 
    num_samples=100,  # Generate 100 fingerprint samples
    save_path='mnist_lenet_cnn_fingerprints.pkl'
)

=== OFFLINE PHASE: Creating Model Fingerprints ===
Generating 100 boundary fingerprint samples...


[W614 07:01:09.896140905 NNPACK.cpp:62] Could not initialize NNPACK! Reason: Unsupported hardware.


Generated 10/100 samples
Generated 20/100 samples
Generated 30/100 samples
Generated 40/100 samples
Generated 50/100 samples
Generated 60/100 samples
Generated 70/100 samples
Generated 80/100 samples
Generated 90/100 samples
Generated 100/100 samples
Successfully generated 100 fingerprint samples
Fingerprints saved to mnist_lenet_cnn_fingerprints.pkl
Created 100 fingerprint samples


In [23]:
# ONLINE PHASE: Test verification with the same model
print("\n" + "="*50)
print("Testing with original model (should show high matching rate):")
is_pirated, matching_rate, results = fingerprinting.verify_model(
    original_model, 
    threshold=0.8,
    fingerprint_path='mnist_lenet_cnn_fingerprints.pkl'
)


Testing with original model (should show high matching rate):
=== ONLINE PHASE: Verifying Suspect Model ===
Testing 100 fingerprint samples...
Processed 20/100 samples
Processed 40/100 samples
Processed 60/100 samples
Processed 80/100 samples
Processed 100/100 samples

=== VERIFICATION RESULTS ===
Total fingerprint samples: 100
Matching predictions: 100
Matching rate: 1.000
Threshold: 0.800
Verdict: PIRATED MODEL DETECTED


In [50]:
# Initialize the suspect model
comp_cnn_model = CompactCNN()

comp_cnn_model.load_state_dict(torch.load('mnist_comp_cnn.pt', map_location=device))
print("Loaded trained model successfully")

Loaded trained model successfully


In [51]:
print("\n" + "="*50)
print("Testing with Independent model:")
is_pirated_diff, matching_rate_diff, results_diff = fingerprinting.verify_model(
    comp_cnn_model,
    threshold=0.8,
    fingerprint_path='mnist_lenet_cnn_fingerprints.pkl'
)


Testing with Independent model:
=== ONLINE PHASE: Verifying Suspect Model ===
Testing 100 fingerprint samples...
Processed 20/100 samples
Processed 40/100 samples
Processed 60/100 samples
Processed 80/100 samples
Processed 100/100 samples

=== VERIFICATION RESULTS ===
Total fingerprint samples: 100
Matching predictions: 83
Matching rate: 0.830
Threshold: 0.800
Verdict: PIRATED MODEL DETECTED


In [6]:
# Initialize the suspect model
mea_model = CompactCNN()

mea_model.load_state_dict(torch.load('mnist_comp_cnn_mea_10.pt', map_location=device))
print("Loaded trained model successfully")

Loaded trained model successfully


In [7]:
print("\n" + "="*50)
print("Testing with MEA 10% model:")
is_pirated_diff, matching_rate_diff, results_diff = fingerprinting.verify_model(
    mea_model,
    threshold=0.8,
    fingerprint_path='mnist_lenet_cnn_fingerprints.pkl'
)


Testing with MEA 10% model:
=== ONLINE PHASE: Verifying Suspect Model ===
Testing 100 fingerprint samples...


[W615 07:19:43.771706053 NNPACK.cpp:62] Could not initialize NNPACK! Reason: Unsupported hardware.


Processed 20/100 samples
Processed 40/100 samples
Processed 60/100 samples
Processed 80/100 samples
Processed 100/100 samples

=== VERIFICATION RESULTS ===
Total fingerprint samples: 100
Matching predictions: 76
Matching rate: 0.760
Threshold: 0.800
Verdict: Model appears legitimate


In [8]:
# Initialize the suspect model
mea_model = CompactCNN()

mea_model.load_state_dict(torch.load('mnist_comp_cnn_mea_30.pt', map_location=device))
print("Loaded trained model successfully")

Loaded trained model successfully


In [9]:
print("\n" + "="*50)
print("Testing with MEA 30% model:")
is_pirated_diff, matching_rate_diff, results_diff = fingerprinting.verify_model(
    mea_model,
    threshold=0.8,
    fingerprint_path='mnist_lenet_cnn_fingerprints.pkl'
)


Testing with MEA 30% model:
=== ONLINE PHASE: Verifying Suspect Model ===
Testing 100 fingerprint samples...
Processed 20/100 samples
Processed 40/100 samples
Processed 60/100 samples
Processed 80/100 samples
Processed 100/100 samples

=== VERIFICATION RESULTS ===
Total fingerprint samples: 100
Matching predictions: 84
Matching rate: 0.840
Threshold: 0.800
Verdict: PIRATED MODEL DETECTED


In [10]:
# Initialize the suspect model
mea_model = CompactCNN()

mea_model.load_state_dict(torch.load('mnist_comp_cnn_mea_50.pt', map_location=device))
print("Loaded trained model successfully")

Loaded trained model successfully


In [11]:
print("\n" + "="*50)
print("Testing with MEA 50% model:")
is_pirated_diff, matching_rate_diff, results_diff = fingerprinting.verify_model(
    mea_model,
    threshold=0.8,
    fingerprint_path='mnist_lenet_cnn_fingerprints.pkl'
)


Testing with MEA 50% model:
=== ONLINE PHASE: Verifying Suspect Model ===
Testing 100 fingerprint samples...
Processed 20/100 samples
Processed 40/100 samples
Processed 60/100 samples
Processed 80/100 samples
Processed 100/100 samples

=== VERIFICATION RESULTS ===
Total fingerprint samples: 100
Matching predictions: 84
Matching rate: 0.840
Threshold: 0.800
Verdict: PIRATED MODEL DETECTED
